In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import math
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
import wandb

from utils.data import get_mnist_data_loaders, get_emnist_data_loaders, randomize_targets, select_from_classes
from utils.visualization import show_imgs, get_model_dot
from utils.others import measure_alloc_mem, count_parameters
from utils.timing import func_timer
from utils.metrics import get_accuracy

from IPython.display import clear_output

torch.set_printoptions(precision=3, linewidth=180)
%env "WANDB_NOTEBOOK_NAME" "main.ipynb"
wandb.login()

In [ ]:
config = {
    "batch_size": 32,
    "targets_noise": 0.,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
}

print(f"... Running on {config['device']} ...")

In [ ]:
train_loader, test_loader, classes = get_mnist_data_loaders(
    batch_size=config["batch_size"], flatten=False, only_classes=None, img_size=28
)
# X, y = next(iter(train_loader))
# X, y = randomize_targets(X, y, noise=config["targets_noise"], n_classes=len(classes))
# show_imgs(X[:10], y[:10].tolist())
# X, y = X.to(config["device"]), y.to(config["device"])

In [ ]:
wandb_run = wandb.init(
    project="", entity="johnny1188", config=config, group="",
    tags=["", ""], notes=f""
)